In [1]:
#-*- coding: utf-8 -*-
import glob
import json

In [2]:
!git clone https://github.com/chinese-poetry/chinese-poetry

Cloning into 'chinese-poetry'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 5801 (delta 1), reused 3 (delta 1), pack-reused 5791
Receiving objects: 100% (5801/5801), 186.29 MiB | 32.09 MiB/s, done.
Resolving deltas: 100% (4502/4502), done.
Checking out files: 100% (1372/1372), done.


In [3]:
!pip install opencc

     |████████████████████████████████| 1.3MB 26.1MB/s 


In [4]:
# 词 is located under ~/chinese-poetry/ci
# 诗 is located under ~/chinese-poetry/json
# poet name could be in 繁體 or 简体 so its better to search for both
import opencc
converter_t2s = opencc.OpenCC('t2s.json')
converter_s2t = opencc.OpenCC('s2t.json')
# 摘出某个诗人的诗
poet_name_simple = "王安石"
poet_name = [poet_name_simple, converter_s2t.convert(poet_name_simple)]
dynasty = "song" # choice from ["tang", "song"]
shiciji_file_name = poet_name_simple
shiciji5 = []
shiciji7 = []
yan = set([])
for p_name in poet_name:
    files = glob.glob("./chinese-poetry/json/poet.{}*.json".format(dynasty))
    #files1 = glob.glob("./chinese-poetry/ci/*.json")
    #files.extend(files1)
    for file in files:
        with open(file) as fi:
            fi_json = json.load(fi)
            for poem in fi_json:
                if 'author' in poem and poem["author"] == p_name:
                    sen_len = len(poem["paragraphs"][0])
                    poem_str = converter_t2s.convert(poem['title'])+":"+converter_t2s.convert("".join(poem["paragraphs"]))
                    if sen_len in [6, 12]: #五言
                      shiciji5.append(poem_str)
                    elif sen_len in [8, 16]: #七言
                      shiciji7.append(poem_str)

print(len(shiciji5))
print(len(shiciji7))
fo5 = open(shiciji_file_name+"5", 'w') #五言
fo7 = open(shiciji_file_name+"7", 'w') #七言
fo = open(shiciji_file_name, 'w') #所有
for item in shiciji5:
    fo5.write(item)
    fo5.write("\n")
    fo.write(item)
    fo.write("\n")
for item in shiciji7:
    fo7.write(item)
    fo7.write("\n")
    fo.write(item)
    fo.write("\n")

1198
2236


In [5]:
!pip install keras

In [6]:
import re
import pandas as pd
import numpy as np
from keras.utils import np_utils
input_file = shiciji_file_name #选择生成的训练集（五言或者7言或者所有）
with open(input_file) as f:
    raw_text = f.read()
lines = raw_text.split("\n")[:-1]
poem_text = [i.split(':')[1] for i in lines]
char_list = [re.findall('[\x80-\xff]{3}|[\w\W]', s) for s in poem_text]
all_words = []
for i in char_list:
    all_words.extend(i)
word_dataframe = pd.DataFrame(pd.Series(all_words).value_counts())
word_dataframe['id']=list(range(1,len(word_dataframe)+1))
word_index_dict = word_dataframe['id'].to_dict()
index_dict = {}
for k in word_index_dict:
    index_dict.update({word_index_dict[k]:k})
seq_len = 2 # 输入长度
rotate_len = 1 # 间隔长度
dataX = []
dataY = []
for sentence in char_list:
  for i in range(0, len(sentence) - seq_len, rotate_len):
      seq_in = sentence[i : i + seq_len]
      seq_out = sentence[i + seq_len]
      dataX.append([word_index_dict[x] for x in seq_in])
      dataY.append(word_index_dict[seq_out])
X = np.array(dataX)
y = np_utils.to_categorical(np.array(dataY))

In [7]:
for i in range(7):
  print([index_dict[word_index] for word_index in dataX[i]])
  print(index_dict[dataY[i]])

['窗', '明']
两
['明', '两']
不
['两', '不']
借
['不', '借']
，
['借', '，']
榻
['，', '榻']
浄
['榻', '浄']
一


In [8]:
import tensorflow as tf

In [9]:
from keras import Sequential
from keras.layers import Embedding, GRU, LSTM, Dense, Activation
import os

def make_model():
  model = Sequential()
  model.add(Embedding(len(word_dataframe)+1, 512))
  #model.add(LSTM(512, return_sequences = True))
  model.add(GRU(512))
  model.add(Dense(y.shape[1]))
  model.add(Activation('softmax'))
  return model

# tf.keras.backend.clear_session()

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

# strategy = tf.distribute.experimental.TPUStrategy(resolver)

# with strategy.scope():
training_model = make_model()
training_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
# define the checkpoint
from keras.callbacks import ModelCheckpoint
import time;
ts = int(time.time())
checkpoint_dir = "./"+input_file+"-checkpoints-"+str(ts)
os.makedirs(checkpoint_dir)
#filepath=checkpoint_dir+"/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint_dir, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
training_model.fit(X, y, epochs=25, batch_size=256, callbacks=callbacks_list)

Epoch 1/25
790/790 [==============================] - 30s 29ms/step - loss: 6.8316

Epoch 00001: loss improved from inf to 6.47807, saving model to ./王安石-checkpoints-1608320091


INFO:tensorflow:Assets written to: ./王安石-checkpoints-1608320091/assets


INFO:tensorflow:Assets written to: ./王安石-checkpoints-1608320091/assets


Epoch 2/25
790/790 [==============================] - 23s 29ms/step - loss: 5.7782

Epoch 00002: loss improved from 6.47807 to 5.65133, saving model to ./王安石-checkpoints-1608320091


INFO:tensorflow:Assets written to: ./王安石-checkpoints-1608320091/assets


...

INFO:tensorflow:Assets written to: ./王安石-checkpoints-1608320091/assets


INFO:tensorflow:Assets written to: ./王安石-checkpoints-1608320091/assets


Epoch 25/25
790/790 [==============================] - 23s 29ms/step - loss: 1.3068

Epoch 00025: loss improved from 1.36773 to 1.36107, saving model to ./王安石-checkpoints-1608320091


INFO:tensorflow:Assets written to: ./王安石-checkpoints-1608320091/assets


INFO:tensorflow:Assets written to: ./王安石-checkpoints-1608320091/assets


In [148]:
import random
import re
from keras.models import load_model
# training_model = make_model()
# training_model.load_weights(filepath)
def gen_poem(seed_text):
    rows = 4 #输出多少句
    cols = 6 #每行多少字，如果五言就6，七言就8
    chars = re.findall('[\x80-\xff]{3}|[\w\W]', seed_text)
    if len(chars) != seq_len:
        return ""
    arr = [word_index_dict[k] for k in chars]
    for i in range(seq_len, rows * cols):
        if (i+1) % cols == 0:
            if (i+1) / cols == 2 or (i+1) / cols == 4:
                arr.append(2)
            else:
                arr.append(1)
        else:
            proba = training_model.predict(np.array(arr[-seq_len:]), verbose=0)
            predicted = np.argsort(proba[1])[-5:]
            index = random.randint(0,len(predicted)-1)
            new_char = predicted[index]
            while new_char == 1 or new_char == 2:
                index = random.randint(0,len(predicted)-1)
                new_char = predicted[index]
            arr.append(new_char)
    poem = [index_dict[i] for i in arr]
    return "".join(poem)


In [149]:
gen_poem("槿花")

'槿花时来伴，何为世情非。此地偏约归，古人间物外。'